In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models, transforms
from PIL import Image
from scipy.ndimage import label
from skimage.measure import regionprops
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage import measure

In [8]:

def global_feature_learning(image):
    # Load a pretrained VGG16 model
    cnn = models.vgg16(pretrained=True).features.eval()

    # Define the image transformations
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Apply the transformations to the image
    image = transform(image).unsqueeze(0)

    # Feed the image into the CNN
    T = cnn(image)
    T = T.unsqueeze(0)
    h, w, d = T.shape

    # Add up the activated tensor through the d direction
    A = T.sum(dim=1)

    # Calculate the mean value of A as the threshold
    delta = A.mean()

    # Generate the mask map A0
    A0 = (A > delta).float()

    # Retain the largest connected component
    A0 = retain_largest_connected_component(A0)

    # Learn the global feature fG
    fG = F.adaptive_avg_pool2d(A0, (1, 1)).view(1, -1)

    return fG

def retain_largest_connected_component(A0):
    # Implement the flood fill algorithm to retain the largest connected component
    labels = measure.label(A0.cpu().numpy())
    largest_label = labels.max()
    if largest_label > 0:
        largest_area = 0
        for i in range(1, largest_label + 1):
            area = (labels == i).sum()
            if area > largest_area:
                largest_area = area
                largest_component = i
        A0[labels != largest_component] = 0
    return A0

In [9]:
# Load an image using PIL
image_path = r'C:\Users\mohit\OneDrive\Desktop\remove_background\entropy\CUB_200_2011\images\001.Black_footed_Albatross\Black_Footed_Albatross_0001_796111.jpg'  # Replace with your image path


In [10]:
from PIL import Image

# Open the image file
image = Image.open(image_path)

In [11]:
global_feature_learning(image)

c:\Users\mohit\.conda\envs\pytorch_1\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mohit\.conda\envs\pytorch_1\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ValueError: too many values to unpack (expected 3)

In [16]:

# Define image transformation
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [17]:
database_images = []
database_filenames = []
import os

In [15]:
database_image_folder = r"C:\Users\mohit\OneDrive\Desktop\remove_background\entropy\CUB_200_2011\images"


In [33]:

# Define image extensions
image_extensions = ['jpg', 'png', 'gif', 'jpeg']

# Iterate over all subdirectories and files in the root directory
for subdir, dirs, files in os.walk(database_image_folder):
    for filename in files:
        # Check if the file is an image
        if filename.split('.')[-1].lower() in image_extensions:
            # Append the filename to the filenames list
            database_filenames.append(filename)
            # Open and transform the image, then append it to the images list
            with Image.open(os.path.join(subdir, filename)) as image:
                image = transform(image)
                database_images.append(image)

# Convert the list of images into a torch tensor
database_images = torch.stack(database_images)


OSError: [Errno 24] Too many open files: 'C:\\Users\\mohit\\OneDrive\\Desktop\\remove_background\\entropy\\CUB_200_2011\\images\\001.Black_footed_Albatross\\Black_Footed_Albatross_0001_796111.jpg'

In [30]:
import glob
def get_image_names_from_folder(root_folder):
    image_extensions = ['jpg', 'png', 'gif', 'jpeg']
    image_names = []
    database_images = []
    for ext in image_extensions:
        image_paths = glob.glob(f'{root_folder}/**/*.{ext}', recursive=True)
        for path in image_paths:
            print(1)
            image_name = os.path.basename(path)
            image = Image.open(path)
            keep = image.copy()
            database_images.append(keep)
            image_names.append(image_name)
            image.close()

    return image_names,database_images

image_names,database_images = get_image_names_from_folder(database_image_folder)

1


OSError: [Errno 24] Too many open files: 'C:\\Users\\mohit\\OneDrive\\Desktop\\remove_background\\entropy\\CUB_200_2011\\images\\001.Black_footed_Albatross\\Black_Footed_Albatross_0001_796111.jpg'

In [23]:
len(image_names)

11788